In [1]:
import pandas as pd
import numpy as np
import warnings

from allensdk.brain_observatory.behavior.behavior_project_cache import (
    VisualBehaviorNeuropixelsProjectCache,
)
import brain_observatory_utilities.datasets.behavior.data_formatting as behavior_utils

import utils

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

%matplotlib inline

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cache_dir = utils.get_data_root()
cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(
    cache_dir=cache_dir, use_static_cache=True
)

In [5]:
ephys_table = cache.get_ecephys_session_table()

In [4]:
session_id = ephys_table.index[12]
print(session_id)
# session = cache.get_ecephys_session(session_id)

1053718935


In [3]:
def distance(data: pd.DataFrame, x_col: str, y_col: str) -> pd.DataFrame:
    """Calculate Euclidean distance between observations of a df.
    
    Parameters
    ----------
    data
        Df where every row is an observation containing an x and y position
    x_col
        Name of column in df containing x positions
    y_col
        Name of column in df containing y positions
    
    Returns
    -------
    distances
        DataFrame of distances between observations
    """
    # Assumes a timestamps column exists and trial_id
    positions = data[[x_col, y_col]]
    positions = positions.diff()
    dist = np.sqrt(np.square(positions[x_col]) + np.square(positions[y_col]))
    distances = pd.DataFrame(dist.values,columns=["distance"])
    distances = distances[~distances["distance"].isna()]
    
    distances["timestamps"] = data["timestamps"]
    distances["trial_id"] = data["trial_id"]
    
    distances = distances[~distances["trial_id"].isna()]
    distances.index = distances["trial_id"]
    distances.index.name = None
    
    return distances

def get_eye_data(session) -> pd.DataFrame:
    # Filtering eye to be in the trial intervals and bin them
    eye = session.eye_tracking
    trials = session.trials
    rewards = session.rewards
    
    bins = pd.concat([pd.Series([0]), trials["stop_time"]])
    eye = eye.loc[(eye["timestamps"] >= trials.iloc[0]["start_time"]) & (eye["timestamps"] <= trials.iloc[-1][ "stop_time"])]
    eye["trial_id"] = pd.cut(eye["timestamps"], bins=bins, labels=trials.index, include_lowest=True, right=True)
    
    avgs = eye.groupby("trial_id").mean()[["pupil_area", "pupil_center_x", "pupil_center_y"]]
    blink_sum = eye.groupby("trial_id").sum()["likely_blink"]
    saccades = distance(eye, "pupil_center_x", "pupil_center_y")
    avg_saccades = saccades.groupby("trial_id").mean()["distance"]
    eye_df = avgs.join(blink_sum).join(avg_saccades).join(rewards["auto_rewarded"])
    eye_df = eye_df.loc[~eye_df["auto_rewarded"].astype(bool)].iloc[:, :-1]
    return eye_df

In [6]:
df = get_eye_data(session)

NameError: name 'session' is not defined

In [ ]:
sns.scatterplot(x=df["pupil_center_x"],y=df["pupil_center_y"], h=trial)
# plt.yscale("log")

In [ ]:
data = pd.concat([utils.session_call(get_eye_data, session_id, "ephys", cache) for session_id in ephys_table.index])

In [ ]:
data = utils.parallel_session_map(get_eye_data, ephys_table.index, "ephys", cores=16)

Using 16 cores.
processing batch 1/3...


In [ ]:
sns.histplot(a["distance"])